# Capstone Week 3 Part 2

#### Importing the library

In [1]:
import pandas as pd

#### Scraping the wiki to get the data

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Renaming the column postal code as shown and correcting the spelling of Neighborhood. This is not the band.

In [3]:
df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
df.head(1)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned


#### Removing rows where **Borough** is **Not assigned**

In [4]:
badrows = df[df['Borough']=='Not assigned'].index
df.drop(badrows, inplace=True)
df.head(12)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### Copying the Borough name to the Neighborhood for the, If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough requirement.

In [5]:
df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df.loc[df['Neighborhood']=='Not assigned'].Borough
df.head(12)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### I use a groupby to combine redundant postal codes and apply a list for each neighborhood. I then use a join to convert eash neighborhood list into a comma delimited string for each row.

In [6]:
df = df.groupby(['PostalCode','Borough'], sort=False, as_index=True)['Neighborhood'].apply(list).reset_index(drop=False)
df['Neighborhood'] = df['Neighborhood'].apply(', '.join)
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### This is the shape of the dataframe.

In [7]:
df.shape

(103, 3)

## Getting the CSV data for Question 2

In [8]:
df1 = pd.read_csv('Geospatial_Coordinates.csv')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Checking the shape

In [9]:
df1.shape

(103, 3)

#### Pre-processing, fixing the column header so it matches the or dataframe and can be used as a key to match the rows in a join.

In [10]:
df1.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df1.head(1)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353


#### In SQL this is a LEFT JOIN, more info on https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html for doing it in pandas. Using validate to check for duplicate keys.

In [12]:
result = pd.merge(df, df1, how='left', on=['PostalCode'], validate="one_to_one")
result.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
